In [ ]:
# Don't forget install this libraries and models!

!pip3 install laserembeddings
!python -m laserembeddings download-models

!pip install -U sentence-transformers

In [ ]:
import os
import re
import csv
import math
import time
import warnings
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Dropout

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from laserembeddings import Laser
laser = Laser()

from sentence_transformers import SentenceTransformer
bert = SentenceTransformer('distiluse-base-multilingual-cased-v2')

warnings.simplefilter("ignore", UserWarning)

In [ ]:
# Train data
print('Read train data...')
en_train = pd.read_csv('hateval2019_en_train.csv')
es_train = pd.read_csv('hateval2019_es_train.csv')
train = pd.concat([en_train, es_train], axis = 0)

print('Load train emdeddings...')
X1 = laser.embed_sentences(train['text'], lang = 'ml') # Uncomment if you want to run model with LASER embeddings
#X1 = bert.encode([text for text in train['text']]) # Uncomment if you want to run model with DistilBERT embeddings
Y1 = to_categorical(train['HS'], 2)
Z1 = to_categorical(train['TR'], 2)


print('Read test data...')
en_test = pd.read_csv('hateval2019_en_dev.csv')
es_test = pd.read_csv('hateval2019_es_dev.csv')
test = pd.concat([en_test, es_test], axis = 0)

print('Load test emdeddings...')
X2 = laser.embed_sentences(test['text'], lang = 'ml') # Uncomment if you want to run model with LASER embeddings
#X2 = bert.encode([text for text in test['text']]) # Uncomment if you want to run model with DistilBERT embeddings
Y2 = to_categorical(test['HS'], 2)
Z2 = to_categorical(test['TR'], 2)

In [ ]:
TRANSFORMER_SHAPE = 1024 # LASER - 1024; DistilBERT — 512

# reshape train data
X1_reshaped = tf.reshape(X1, [-1, 1, TRANSFORMER_SHAPE])
X2_reshaped = tf.reshape(X2, [-1, 1, TRANSFORMER_SHAPE])
Y1_reshaped = tf.reshape(Y1, [-1, 1, 2])
Y2_reshaped = tf.reshape(Y2, [-1, 1, 2])
Z1_reshaped = tf.reshape(Z1, [-1, 1, 2])
Z2_reshaped = tf.reshape(Z2, [-1, 1, 2])

In [ ]:
# evaluation function

def evaluate(pred, gold):
    """
    evaluate accuracy and macro-F1 of ABSA task
    """
    pred_count = np.zeros(3, dtype='int32')
    gold_count = np.zeros(3, dtype='int32')
    hit_count = np.zeros(3, dtype='int32')

    # number of testing documents
    n_test = len(gold)
    error_cases = {}
    for i in range(n_test):
        y_p = int(pred[i])
        y_g = gold[i]
        pred_count[y_p] += 1
        gold_count[y_g] += 1
        if y_p == y_g:
            hit_count[y_p] += 1
        else:
            error_cases[i] = [y_p, y_g]
    # number of true predictions
    total_hit = sum(hit_count)
    # accuracy
    acc = float(total_hit) / n_test
    # macro_f1
    macro_f = f1_score(y_true=gold, y_pred=pred, labels=[0, 1], average='weighted')
    # auc
    auc = roc_auc_score(gold, pred)

    result_string = ''
    result_string = '%sneg: recall: %s/%s, precision: %s/%s \n' % (result_string,
                                                                   hit_count[0], gold_count[0], hit_count[0],
                                                                   pred_count[0])
    result_string = '%spos: recall: %s/%s, precision: %s/%s \n' % (result_string,
                                                                   hit_count[1], gold_count[1], hit_count[1],
                                                                   pred_count[1])
    result_string = '%sneu: recall: %s/%s, precision: %s/%s \n' % (result_string,
                                                                   hit_count[2], gold_count[2], hit_count[2],
                                                                   pred_count[2])
    return acc, auc, macro_f, result_string, error_cases

In [ ]:
DROPOUT = 0.1
batch_size = 128 
n_epoch = 20
learning_rate = 0.01

class MLIAN(tf.keras.Model):
    def __init__(self):
        super(MLIAN, self).__init__()

        self.n_hidden = 300
        self.n_class = 2
        self.l2_reg = 0.00001

        self.max_aspect_len = TRANSFORMER_SHAPE
        self.max_context_len = TRANSFORMER_SHAPE

        self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)

        self.aspect_lstm = tf.keras.layers.LSTM(self.n_hidden,
                                                return_sequences=True,
                                                recurrent_initializer='glorot_uniform', # check this glorot_uniform
                                                stateful=True)
        self.context_lstm = tf.keras.layers.LSTM(self.n_hidden,
                                                return_sequences=True,
                                                recurrent_activation='sigmoid', # check this
                                                recurrent_initializer='glorot_uniform', # check this glorot_uniform
                                                stateful=True)
        self.aspect_w = tf.Variable(tf.random.normal([self.n_hidden, self.n_hidden]), name='aspect_w')
        self.aspect_b = tf.Variable(tf.zeros([self.n_hidden]), name='aspect_b')

        self.context_w = tf.Variable(tf.random.normal([self.n_hidden, self.n_hidden]), name='context_w')
        self.context_b = tf.Variable(tf.zeros([self.n_hidden]), name='context_b')

        self.output_fc = tf.keras.layers.Dense(self.n_class, kernel_regularizer=tf.keras.regularizers.l2(l=self.l2_reg)) # check l2

    def call(self, data, dropout=DROPOUT):
        aspects, contexts, labels = data
        # aspects = Target parameter
        aspect_inputs = tf.cast(aspects, tf.float32)
        aspect_inputs = tf.nn.dropout(aspect_inputs, dropout)

        # contexts = laser embeddings
        context_inputs = tf.cast(contexts, tf.float32)
        context_inputs = tf.nn.dropout(context_inputs, dropout)

        aspect_outputs = self.aspect_lstm(aspect_inputs)
        aspect_avg = tf.reduce_mean(aspect_outputs, 1)

        context_outputs = self.context_lstm(context_inputs)
        context_avg = tf.reduce_mean(context_outputs, 1)

        aspect_att = tf.nn.softmax(tf.math.tanh(tf.einsum('ijk,kl,ilm->ijm', aspect_outputs, self.aspect_w, # check this tf.nn.tanh
                                                        tf.expand_dims(context_avg, -1)) + self.aspect_b), axis=1)

        aspect_rep = tf.reduce_sum(aspect_att * aspect_outputs, 1)
        context_att = tf.nn.softmax(tf.math.tanh(tf.einsum('ijk,kl,ilm->ijm', context_outputs, self.context_w, # check this tf.nn.tanh
                                                         tf.expand_dims(aspect_avg, -1)) + self.context_b),
                                    axis=1)
        context_rep = tf.reduce_sum(context_att * context_outputs, 1)

        rep = tf.concat([aspect_rep, context_rep], 1)
        predict = self.output_fc(rep)

        return predict, labels

def run(model, train_data, test_data):
    print('Training ...')
    max_acc, max_f1, max_auc, step = 0., 0., 0., -1
    logdir = '/logs/'

    train_data_size = len(train_data[0])
    #print(train_data)
    train_data = tf.data.Dataset.from_tensor_slices(train_data)
    train_data = train_data.shuffle(buffer_size=train_data_size).batch(batch_size, drop_remainder=True)

    test_data_size = len(test_data[0])
    test_data = tf.data.Dataset.from_tensor_slices(test_data)
    test_data = test_data.batch(batch_size, drop_remainder=True)


    iterator = tf.compat.v1.data.Iterator.from_structure(tf.compat.v1.data.get_output_types(train_data),
                                           tf.compat.v1.data.get_output_shapes(train_data))
    optimizer = tf.optimizers.Adamax(learning_rate=learning_rate)
    writer = tf.summary.create_file_writer(logdir)
    writer.set_as_default()

    for i in range(n_epoch):
        cost, predict_list, labels_list = 0., [], []
        iterator.make_initializer(train_data)
        for _ in range(math.floor(train_data_size / batch_size)):
            data = iterator.get_next()
            with tf.GradientTape() as tape:
                predict, labels = model(data, dropout=DROPOUT)
                loss_t = tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=labels)
                loss = tf.reduce_mean(loss_t)
                cost += tf.reduce_sum(loss_t)
            grads = tape.gradient(loss, model.variables)
            optimizer.apply_gradients(zip(grads, model.variables))
            predict_list.extend(tf.argmax(tf.nn.softmax(predict), 1).numpy())
            labels_list.extend(tf.argmax(labels, 1).numpy())
        train_acc, train_auc, train_f1, _, _ = evaluate(pred=predict_list, gold=labels_list)
        train_loss = cost / train_data_size
        tf.summary.scalar('train_loss', train_loss, step=step)
        tf.summary.scalar('train_acc', train_acc, step=step)
        tf.summary.scalar('train_auc', train_auc, step=step)
        tf.summary.scalar('train_f1', train_f1, step=step)

        cost, predict_list, labels_list = 0., [], []
        iterator.make_initializer(test_data)
        for _ in range(math.floor(test_data_size / batch_size)):
            data = iterator.get_next()
            predict, labels = model(data, dropout=DROPOUT)
            loss_t = tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=labels)
            cost += tf.reduce_sum(loss_t)
            predict_list.extend(tf.argmax(tf.nn.softmax(predict), 1).numpy())
            labels_list.extend(tf.argmax(labels, 1).numpy())
        test_acc, test_auc, test_f1, _, _ = evaluate(pred=predict_list, gold=labels_list)
        test_loss = cost / test_data_size

        tf.summary.scalar('test_loss', test_loss, step=step)
        tf.summary.scalar('test_acc', test_acc, step=step)
        tf.summary.scalar('test_auc', test_auc, step=step)
        tf.summary.scalar('test_f1', test_f1, step=step)

        if test_acc + test_auc + test_f1 > max_acc + max_auc + max_f1:
            max_acc = test_acc
            max_auc = test_auc
            max_f1 = test_f1
            step = i
        print('epoch %s: train-loss=%.6f; train-acc=%.6f; train-auc=%.6f; train-f1=%.6f; test-loss=%.6f; test-acc=%.6f; test-auc=%.6f; test-f1=%.6f.' % (
                str(i), train_loss, train_acc, train_auc, train_f1, test_loss, test_acc, test_auc, test_f1))

    print('The max accuracy of testing results: acc %.6f, auc %.6f and f1-wighted %.6f of step %s' % (max_acc, max_auc, max_f1, step))

In [ ]:
print('Loading training data and testing data ...')

train_data = Z1_reshaped, X1_reshaped, np.asarray(Y1).astype('float32')
test_data = Z2_reshaped, X2_reshaped, np.asarray(Y2).astype('float32')

print('Loading model ...')
model = MLIAN()
start_time = time.time()

run(model, train_data, test_data)
end_time = time.time()
print('Time Costing: %s' % (end_time - start_time))